In [1]:
from IPython.core.display import Markdown, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import traceback
import numpy as np
import scipy.stats as ss
import yaml
import itertools
import sys
import os
import subprocess
from os import path
import pickle
import pandas as pd
import matplotlib.patches as patches
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.linear_model import LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

def printmd(*args):
    display(Markdown(' '.join(map(str, args))))

In [2]:
# !ros2 run local_planning_performance_modelling compute_metrics -r "~/ds/performance_modelling/output/test_local_planning/*"

In [3]:
# !ros2 run local_planning_performance_modelling compute_metrics -r "~/ds/performance_modelling/output/test_local_planning_gen_circles/*" -s

In [4]:
local_planner_color = {'dwb': 'blue', 'rpp': 'orange', 'teb': 'green', 'rpp_bad': 'yellow'}
robot_model_color = {'hunter': 'cyan', 'turtle': 'green'}
pd.options.display.width = 500
pd.options.display.max_rows = 0
pd.options.display.float_format = '{:,.3f}'.format
plt.rcParams['figure.figsize'] = [18, 18]
plt.rcParams['lines.linewidth'] = 2
# plt.rcParams['lines.markersize'] = 3
# plt.rcParams['lines.marker'] = 'x'
plt.rcParams['scatter.marker'] = '.'

# fg_color = 'white'
# plt.rcParams['grid.color'] = 'gray'
# plt.rcParams['text.color'] = fg_color
# plt.rcParams['ytick.color'] = fg_color
# plt.rcParams['xtick.color'] = fg_color
# plt.rcParams['axes.labelcolor'] = fg_color
# plt.rcParams['axes.edgecolor'] = fg_color

# bg_color = "#323a48"
# plt.rcParams['figure.facecolor'] = bg_color
# plt.rcParams['axes.facecolor'] = bg_color
# plt.rcParams['legend.facecolor'] = bg_color

In [5]:
# df_real = pd.read_csv(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results.csv"))
# with open(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results_info.yaml")) as results_real_info_file:
#     results_real_info = yaml.safe_load(results_real_info_file)
# df_real = df_real[(df_real.amcl_alpha_factor.notna()) & (df_real.robot_model=='turtlebot3_waffle_performance_modelling')]
# df = df_real

# for i in range(0, 4):
#     df[f'beta_{i+1}'] = df['odometry_error'].apply(lambda x: eval(x)[i])

# df.rename(inplace=True, columns={
#     'collisionless_localization_update_absolute_translation_error_mean': 'absolute_translation_error',
#     'collisionless_localization_update_absolute_rotation_error_mean': 'absolute_rotation_error',
#     'collisionless_localization_update_normalized_relative_translation_error_mean': 'normalized_relative_translation_error',
#     'collisionless_localization_update_normalized_relative_rotation_error_mean': 'normalized_relative_rotation_error',
#     'localization_update_rate_mean': 'localization_update_rate',
# })
# df.loc[df.robot_model == 'turtlebot3_waffle_performance_modelling', 'robot_model'] = 'turtle'
# df.loc[df.robot_model == 'hunter2', 'robot_model'] = 'hunter'
# df['session_id'] =  df['run_id'].apply(lambda x:  x.split('_')[1]+'_'+x.split('_')[2]+'_'+x.split('_')[3])
# df['run_number'] =  df['run_id'].apply(lambda x:  int(x.split('_')[5]))
# df.max_steering_angle_deg = df.max_steering_angle_deg.fillna(90)

# # add aggregated data
# min_trajectory_length_group_df = df.groupby(["environment_name", "run_index", "collisionless_success_rate"])
# for (environment_name, run_index, collisionless_success_rate), group_df in min_trajectory_length_group_df:
#     df.loc[(df.environment_name == environment_name) & (df.run_index == run_index) & (collisionless_success_rate), 'min_trajectory_length'] = group_df.trajectory_length.min()
# df['norm_trajectory_length'] = df.trajectory_length / df.min_trajectory_length
# metrics += ['norm_trajectory_length']
# metrics_and_versions += ['norm_trajectory_length']

# min_execution_time_group_df = df.groupby(["environment_name", "run_index", "collisionless_success_rate"])
# for (environment_name, run_index, collisionless_success_rate), group_df in min_execution_time_group_df:
#     df.loc[(df.environment_name == environment_name) & (df.run_index == run_index) & (collisionless_success_rate), 'min_execution_time'] = group_df.execution_time.min()
# df['norm_execution_time'] = df.execution_time / df.min_execution_time
# metrics += ['norm_execution_time']
# metrics_and_versions += ['norm_execution_time']

# df.loc[(df.fixed_rpp != True) & (df.local_planner_node == 'rpp'), 'local_planner_node'] = 'rpp_bad'
# df = df[df.local_planner_node != 'rpp_bad'].copy()

# df = df[(df.max_steering_angle_deg == 90.) & (df.robot_model == 'turtle') & (df.global_planner_node == 'navfn') & (df.localization_node == 'amcl')].copy()

In [6]:

df_real = pd.read_csv(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results.csv"))
with open(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results_info.yaml")) as results_real_info_file:
    results_real_info = yaml.safe_load(results_real_info_file)
df_real = df_real[(df_real.amcl_alpha_factor.notna()) & (df_real.robot_model=='turtlebot3_waffle_performance_modelling')]

df_gen = pd.read_csv(path.expanduser("~/ds/performance_modelling/output/test_local_planning_gen_circles/results.csv"))
with open(path.expanduser("~/ds/performance_modelling/output/test_local_planning_gen_circles/results_info.yaml")) as results_gen_info_file:
    results_gen_info = yaml.safe_load(results_gen_info_file)

df = df_gen.append(df_real, sort=True)

df.rename(inplace=True, columns={
    'collisionless_localization_update_absolute_translation_error_mean': 'absolute_translation_error',
    'collisionless_localization_update_absolute_rotation_error_mean': 'absolute_rotation_error',
    'collisionless_localization_update_normalized_relative_translation_error_mean': 'normalized_relative_translation_error',
    'collisionless_localization_update_normalized_relative_rotation_error_mean': 'normalized_relative_rotation_error',
    'localization_update_rate_mean': 'localization_update_rate',
})

results_info = results_gen_info
results_info['run_parameter_names'] += [i for i in results_real_info['run_parameter_names'] if i not in results_info['run_parameter_names']]

# TEMP: only consider real localization
df = df[df.localization_node != 'localization_generator']
# df = df[df.amcl_alpha_factor == 1.0]

df.fixed_rpp = df.fixed_rpp.fillna(False)
df.loc[(df.fixed_rpp == False) & (df.local_planner_node == 'rpp'), 'local_planner_node'] = 'rpp_bad'
df = df[df.local_planner_node != 'rpp_bad'].copy()

# df = df[df.amcl_alpha_factor.notna()]

# turn odometry_error into beta_1..4
results_info['run_parameter_names'] += ['beta_1', 'beta_2', 'beta_3', 'beta_4']
results_info['run_parameter_names'].remove('odometry_error')
for i in range(0, 4):
    df[f'beta_{i+1}'] = df['odometry_error'].apply(lambda x: eval(x)[i])
del df['odometry_error']

df.loc[df.robot_model == 'turtlebot3_waffle_performance_modelling', 'robot_model'] = 'turtle'
df.loc[df.robot_model == 'hunter2', 'robot_model'] = 'hunter'

run_parameters = [c for c in list(df.columns) if c in results_info['run_parameter_names']]
metrics_versions = [c for c in list(df.columns) if '_version' in c]
everything_else = ['run_id', 'session_id', 'run_number']
metrics = [c for c in df.columns if c not in metrics_versions + run_parameters + everything_else]
metrics_and_versions = [c for c in list(df.columns) if '_version' in c or c in metrics]

cpu_time_metrics = [c for c in metrics if 'cpu_time' in c]
max_memory_metrics = [c for c in metrics if 'max_memory' in c]

# add useful parameters
df['session_id'] =  df['run_id'].apply(lambda x:  x.split('_')[1]+'_'+x.split('_')[2]+'_'+x.split('_')[3])
df['run_number'] =  df['run_id'].apply(lambda x:  int(x.split('_')[5]))
df[max_memory_metrics] = df[max_memory_metrics]/1024**2
df.max_steering_angle_deg = df.max_steering_angle_deg.fillna(90)

# add metrics from existing ones
df['average_velocity'] = df['trajectory_length'] / df['execution_time']
df['collisionless_success_rate'] = df['success_rate'] & (1 - df['collision_rate'])
metrics += ['average_velocity', 'collisionless_success_rate']
metrics_and_versions += ['average_velocity', 'collisionless_success_rate']

# add aggregated data
min_trajectory_length_group_df = df.groupby(["environment_name", "run_index", "collisionless_success_rate"])
for (environment_name, run_index, collisionless_success_rate), group_df in min_trajectory_length_group_df:
    df.loc[(df.environment_name == environment_name) & (df.run_index == run_index) & (collisionless_success_rate), 'min_trajectory_length'] = group_df.trajectory_length.min()
df['norm_trajectory_length'] = df.trajectory_length / df.min_trajectory_length
metrics += ['norm_trajectory_length']
metrics_and_versions += ['norm_trajectory_length']

min_execution_time_group_df = df.groupby(["environment_name", "run_index", "collisionless_success_rate"])
for (environment_name, run_index, collisionless_success_rate), group_df in min_execution_time_group_df:
    df.loc[(df.environment_name == environment_name) & (df.run_index == run_index) & (collisionless_success_rate), 'min_execution_time'] = group_df.execution_time.min()
df['norm_execution_time'] = df.execution_time / df.min_execution_time
metrics += ['norm_execution_time']
metrics_and_versions += ['norm_execution_time']

df = df[(df.max_steering_angle_deg == 90.) & (df.robot_model == 'turtle') & (df.global_planner_node == 'navfn') & (df.localization_node == 'amcl') & ((df.amcl_alpha_factor == 1.0) | (df.amcl_alpha_factor == 0.0))]

if False:
    printmd("## Number of Runs")
    printmd(f"    {len(df.run_id.unique())}")

    printmd("## Run Parameters")
    for name in [run_parameter_name for run_parameter_name in run_parameters if 'localization_generator_' not in run_parameter_name]:
        values = list(df[name].unique())
        printmd(f"    {name:<70}", sorted(values))

    printmd("## Metrics")
    for name in metrics_and_versions:
        if name in metrics_versions:
            if len(df[name].unique()) == 1:
                printmd(f"    {name:<70} {sorted(df[name].unique())}")
            else:
                printmd(f"<code><font style='background-color:yellow;font-family:monospace'>{name:<70}{sorted(df[name].unique())} </font></code> ⚠️")
        else:
            printmd(f"    {name:<70} min: {df[name].min(skipna=True):10.4g} {'avg':>15}: {df[name].mean(skipna=True):10.4g} {'max':>15}: {df[name].max(skipna=True):10.4g} {'nan':>15}: {sum(df[name].isna()):10.4g}")

## Number of Runs

    1093

## Run Parameters

    amcl_alpha_factor                                                      [0.0, 1.0]

    environment_name                                                       ['7A-2', 'airlab', 'fr079', 'intel', 'mexico', 'office_b']

...

    translation_rotation_product                                           min:   0.002615             avg:    0.02795             max:     0.1523             nan:         56

    average_velocity                                                       min:          0             avg:     0.2194             max:     0.2681             nan:          0

    collisionless_success_rate                                             min:          0             avg:     0.9048             max:          1             nan:          0

    norm_trajectory_length                                                 min:          0             avg:      0.978             max:      1.675             nan:          0

    norm_execution_time                                                    min:  0.0009367             avg:      1.034             max:      2.031             nan:          0

In [7]:
if False:
    base_path = "/home/enrico/ds/performance_modelling/output/test_local_planning/"
    out_dir = "thesis_plots/localization_time/"
    if not path.exists(out_dir):
        os.makedirs(out_dir)

    run_indices = df.run_index.unique()
    environment_names = df.environment_name.unique()
    local_planners = df.local_planner_node.unique()

    max_scatter_points = 100
    for environment_name in environment_names:
        for run_index in run_indices:
    #         print()
    #         print(environment_name, run_index)
            df_s = df[(df.environment_name == environment_name) & (df.run_index == run_index)].copy()
            for local_planner in local_planners:
                run_ids = df_s[df_s.local_planner_node == local_planner].run_id
                run_output_folders = list(map(lambda x: path.join(base_path, x), run_ids))
                for i, run_output_folder in enumerate(run_output_folders):
                    # check the run is valid from run events
                    run_events_file_path = path.join(run_output_folder, "benchmark_data", "run_events.csv")
                    run_events_df = pd.read_csv(run_events_file_path, engine='python', sep=', ')
                    navigation_start_events = run_events_df[run_events_df.event == 'navigation_goal_accepted']
                    navigation_succeeded_events = run_events_df[(run_events_df.event == 'navigation_succeeded')]
                    navigation_failed_events = run_events_df[(run_events_df.event == 'navigation_failed')]
                    if len(navigation_start_events) != 1 or len(navigation_succeeded_events) + len(navigation_failed_events) != 1:
                        continue

                    # get the dataframes for ground truth poses
                    ground_truth_poses_file_path = path.join(run_output_folder, "benchmark_data", "ground_truth_poses.csv")
                    ground_truth_poses_df = pd.read_csv(ground_truth_poses_file_path)                
                    plt.scatter(ground_truth_poses_df.v_theta, ground_truth_poses_df.v_x, marker='.', s=1**2, label=local_planner if i == 0 else None, color=local_planner_color[local_planner], alpha=0.5)#, rasterized=True)

            rect=patches.Rectangle((-max_vel_theta, -max_vel_x), 2*max_vel_theta, 2*max_vel_x, fill=False, color="black", linewidth=1, alpha=0.5)
            plt.gca().add_patch(rect)

            plt.xlabel('v_theta')
            plt.ylabel('v_x')
            plt.xlim([-2*max_vel_theta, 2*max_vel_theta])
            plt.ylim([-2*max_vel_x, 2*max_vel_x])
            plt.grid()
            plt.legend()
#             plt.savefig(path.join(out_dir, f"vel_theta_x_distribution_{environment_name}_{run_index}.png"), bb_inches='tight')
            plt.show()
            plt.close()